Here we will load an LFRic NetCDF output file and plot some fields from the model output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import iris
import geovista as gv
import pyvista as pv
import netCDF4 as nc
import os

First, let's load the mesh file, in order to get the connectivity information between the faces for plotting.

In [ ]:
mesh_path = '/PATH/TO/MY/MESH/FILE/'
# mesh_path = '~/cylc-run/my_lfric_dir/run1/share/output/meshes/intel_fast-debug-64bit/'
#
# You may have executed the test on another machine and may need to use scp to transfer files in order to plot.
# scp MACHINE:~/cylc-run/my_lfric_dir/run1/share/output/meshes/intel_fast-debug-64bit/mesh_C24_MG.nc .
# mesh_path = os.getcwd()

mesh_data = nc.Dataset(mesh_path + '/' + 'mesh_C24_MG.nc','r')
face_conn = mesh_data['dynamics_face_links'][:]

Now, let's load the model output file, and check the header info.

In [ ]:
file_path = '/PATH/TO/MY/OUTPUT/FILE/'
# file_path = '~/cylc-run/my_lfric_dir/run1/share/output/gungho_model/gungho_model_baroclinic-C24_MG_xc40_intel_fast-debug-64bit/results/'
#
# You may have executed the test on another machine and may need to use scp to transfer files in order to plot.
# scp MACHINE:~/cylc-run/my_lfric_dir/run1/share/output/gungho_model/gungho_model_baroclinic-C24_MG_xc40_intel_fast-debug-64bit/results/lfric_diag.nc .
# file_path = os.getcwd()

data = nc.Dataset(file_path + '/' + 'lfric_diag.nc','r')
print(data)

OK, now from the field names in the header info above, let's pick a field to view, maybe the Exner pressure?

In [ ]:
field_name = 'exner'
field = data[field_name][:]
print(field.shape)

The first dimension is the number of time dumps, the second the number of vertical levels, and the third is the number of values on the faces of the elements at each level. Let's look at the botttom level at the final time.

In [ ]:
time = field.shape[0]-1
level = 0
field_2d = field[time,level,:]

Now let's plot the final time, bottom level of the field you've selected. For this we'll need to the 2D coordinates on the sphere (longitude, latitude), the field data array and the face connectivity from the mesh file. We'll also add the coastlines at the element edges just to be fancy :)

In [ ]:
face_lon = data['Mesh2d_face_x'][:]
face_lat = data['Mesh2d_face_y'][:]
gv_mesh = gv.Transform.from_unstructured(face_lon, face_lat, connectivity=face_conn, data=field_2d)

plotter = gv.GeoPlotter()
plotter.add_mesh(gv_mesh, cmap="nipy_spectral", show_edges=True)
plotter.add_coastlines(color='k')
plotter.camera.zoom(1.6)
viewpoint=[(1.6839421754488628, 4.701586250713391, 4.454449119537074), (0.0, 0.0, 0.0), (-0.2577906080031461, -0.6142650898541594, 0.7458031924119324)]
plotter.camera_position=viewpoint
plotter.show()